In [1]:
#Libraries
import requests
from bs4 import BeautifulSoup as soup
import pgeocode
from random import randint
from time import sleep
import numpy as np
import csv
import re

We found that to move in between pages of restaurants in TripAdvisor they have the following path

In [1]:
p='40'
'https://www.tripadvisor.com.mx/RestaurantSearch-g150799-oa'+p+'-Guanajuato_Central_Mexico_and_Gulf_Coast.html#EATERY_LIST_CONTENTS'

'https://www.tripadvisor.com.mx/RestaurantSearch-g150799-oa40-Guanajuato_Central_Mexico_and_Gulf_Coast.html#EATERY_LIST_CONTENTS'

where p stands for the decimal number of pages, say, 10 means the page displays the first 30 restaurants. In total, there are 325 restaurants in Guanajuato, Mex. 

The use of headers is not request the page in a nicely way

In [23]:
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

The following functions were created to help

In [2]:
def get_links_restaurants(bsop): 
    links=[]
    for review in bsop.findAll('a',{'class':'Lwqic Cj b'}):
        a=review['href']
        a='https://www.tripadvisor.com.mx'+a
        aux=a.find('Reviews')
        a= a[:(aux+7)]+a[(aux+7):]
        links.append(a)
    return links

def get_data(bsop):
    name=bsop.find('h1',{'data-test-target':'top-info-header'}).get_text()
    tipo=[x.get_text() for x in bsop.findAll('div',{'class':'SrqKb'})]
    n_rev=bsop.find('a',{'class':'IcelI'}).get_text()[0]
    ran=bsop.find('span',{'class':'ZDEqb'}).get_text()[0]
    adress=bsop.find('div',{'class':'kDZhm IdiaP Me'}).get_text()
    return [name,tipo,int(n_rev),int(ran),adress]

The info we scrapped is store in a restaurants.csv file.

In [ ]:
path_to_file = "./restaurants.csv"
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

In [27]:
i=0
for p in ['10', '40', '70','100', '140','170','200','230','260','290','320']:
    sleep(randint(1,5))
    url_='https://www.tripadvisor.com.mx/RestaurantSearch-g150799-oa'+p+'-Guanajuato_Central_Mexico_and_Gulf_Coast.html#EATERY_LIST_CONTENTS'
    html=requests.get(url_, headers=headers)
    bsop=soup(html.content,'lxml')
    links=get_links_restaurants(bsop)
    for link in links:
        print(link)
        idx=int(re.findall(r'\d+[1-9]+',link)[1])
        #call the server
        html_temp=requests.get(link, headers=headers)
        bsop_temp=soup(html_temp.content,'lxml')
        print(i,[idx]+get_data(bsop_temp))
        csvWriter.writerow([idx]+get_data(bsop_temp))
        i=i+1

https://www.tripadvisor.com.mx/Restaurant_Review-g151932-d23014476-Reviews-Ruby_Joy_s_Delivery_Only_Restaurant-San_Miguel_de_Allende_Central_Mexico_and_Gul.html
0 [23014476, "Ruby Joy's Delivery Only Restaurant", ['$119 - $298', 'Contemporánea, Saludable, Internacional, Delicatessen, Fusión', 'Almuerzo, Cena'], 2, 5, 'Delivery Only, San Miguel de Allende 37796 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7291750-Reviews-La_Vie_en_Rose-Guanajuato_Central_Mexico_and_Gulf_Coast.html
1 [729175, 'La Vie en Rose', ['$45 - $300', 'Francesa, Café, Europea', 'Apto para vegetarianos, Opciones veganas'], 3, 4, 'Cantarranas # 18, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d2052606-Reviews-Santo_Cafe-Guanajuato_Central_Mexico_and_Gulf_Coast.html
2 [2052606, 'Santo Cafe', ['$75 - $250', 'Mexicana, Café', 'Apto para vegetarianos, Opciones veganas, Opciones sin gluten'], 3, 4, 'Campanero 4, Guanajuato 36000 México']
https://www.tripadvisor.

25 [4739294, 'A Punto Restaurante-Delicatessen', ['Europea, Mediterránea', 'Apto para vegetarianos', 'Cena, Almuerzo'], 6, 4, '"Casa Cuatro"  San Jose #4   Centro Historico, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7654484-Reviews-Escarola-Guanajuato_Central_Mexico_and_Gulf_Coast.html
26 [7654484, 'Escarola', ['Saludable, Mexicana', 'Apto para vegetarianos, Opciones veganas, Opciones sin gluten', 'Desayuno, Almuerzo, Brunch, Cena'], 9, 4, 'Calle Positos No. 38 Centro, Frente a Casa Museo Diego Rivera, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d8358992-Reviews-Bar_Tradicional_Luna-Guanajuato_Central_Mexico_and_Gulf_Coast.html
27 [8358992, 'Bar Tradicional  Luna', ['Mexicana, Bar', 'Cena, Brunch, Bebidas, Almuerzo', 'Televisión, Sirve alcohol, Bar completo, Vino y cerveza, Acepta American Express, Acepta Mastercard, Acepta Visa, Pagos digitales, Solo pago en efectivo, Wi-Fi gratis, Asientos al aire libre,

50 [23445451, 'Nigromante Roftoop', ['Bar, Mediterránea, Pub', 'Reservaciones'], 9, 5, 'Positos #79, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d11893364-Reviews-La_Champa-Guanajuato_Central_Mexico_and_Gulf_Coast.html
51 [11893364, 'La Champa', ['$2,386 - $3,976', 'Cena, Abierto hasta tarde, Bebidas', 'Asiento, Sillitas altas disponibles, Sirve alcohol, Bar completo, Vino y cerveza, Acepta tarjetas de crédito'], 1, 4, 'Baratillo 9 Plaza Baratillo 9 Col Centro, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d14923558-Reviews-Las_Vieyras_Restaurante-Guanajuato_Central_Mexico_and_Gulf_Coast.html
52 [14923558, 'Las Vieyras Restaurante', ['$10 - $700', 'Internacional, Saludable', 'Desayuno, Almuerzo, Cena, Brunch, Abierto hasta tarde, Bebidas'], 2, 4, 'Calle Positos 33, Col. Centro. C.P. 36000 Guanajuato, Gto., Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7233466-Reviews-La_Inu

75 [21347622, 'Sierra BBQ Santa Rosa de Lima', ['Almuerzo, Cena'], 6, 5, 'Kilometer 13 Santa Rosa de Lima, Guanajuato 36220 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7352011-Reviews-PACIFICO_Surf-Guanajuato_Central_Mexico_and_Gulf_Coast.html
76 [7352011, 'PACIFICO Surf', ['$5 - $150', 'Mariscos, Mexicana, Comida rápida', 'Almuerzo, Cena'], 2, 4, 'Manuel Doblado Number 16, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d15345898-Reviews-Vamos_A_Ver_A_Cruz-Guanajuato_Central_Mexico_and_Gulf_Coast.html
77 [15345898, 'Vamos A Ver A Cruz', ['Mariscos'], 9, 4, 'Calle Alonso #66 Altos, Col. Centro Casi Esquina Con Calle Juárez, Guanajuato 37000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d8451028-Reviews-El_Rincon_de_Fausto-Guanajuato_Central_Mexico_and_Gulf_Coast.html
78 [8451028, 'El Rincon de Fausto', ['Mexicana', 'Desayuno, Brunch', 'Reservaciones, Servicio de mesa, Asiento'], 1, 4, 'Jardín reforma 15 a, Gu

101 [7316951, 'Tierra de Maiz', ['Mexicana', 'Cena'], 9, 4, 'Olas 3120 Local B, Guanajuato 3660 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7597249-Reviews-Pacifico_Surf-Guanajuato_Central_Mexico_and_Gulf_Coast.html
102 [7597249, 'Pacifico Stop&Go', ['$25 - $70', 'Comida rápida, Saludable, Mariscos', 'Almuerzo, Cena'], 7, 4, 'Guanajuato, Gto. Avenida Manuel Doblado 16, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d23991551-Reviews-La_Nona_Lola_Gto-Guanajuato_Central_Mexico_and_Gulf_Coast.html
103 [23991551, 'La Nona Lola Gto', ['Italiana, Mexicana', 'Almuerzo, Cena'], 6, 5, 'Constancia 10 Col. Centro A Un Costado Del Estacionamiento Alonso., Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d16724743-Reviews-Bollos_Prenaos-Guanajuato_Central_Mexico_and_Gulf_Coast.html
104 [16724743, 'Bollos Preñaos', ['Mexicana, Café, Española', 'Desayuno, Almuerzo, Brunch, Bebidas', 'Comida para llevar, Asien

...

156 [9818093, 'Cantina La Botellita', ['Mexicana', 'Comida para llevar, Asiento, Servicio de mesa'], 3, 3, 'Calle Jardin de La Union 2, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d12671034-Reviews-Gorditas_y_Tlacoyos_de_San_Roque-Guanajuato_Central_Mexico_and_Gulf_Coast.html
157 [12671034, 'Gorditas y Tlacoyos de San Roque', [], 2, 5, 'Callejón Cantaritos, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d11995832-Reviews-El_Minero-Guanajuato_Central_Mexico_and_Gulf_Coast.html
158 [11995832, 'El Minero', [], 2, 5, 'Alhondiga 12A, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d11983069-Reviews-La_Taberna_de_Sancho-Guanajuato_Central_Mexico_and_Gulf_Coast.html
159 [11983069, 'La Taberna de Sancho', ['Almuerzo, Cena'], 6, 3, 'Plaza de La Paz #38, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d2090983-Reviews-La_Casa_Restaurante-Guanajuato_Cent

187 [6001489, 'La Terraza Del Quijote', ['Española', 'Desayuno, Almuerzo'], 3, 4, 'Carretera Guanajuato a Dolores Hidalgo, Guanajuato México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d12509850-Reviews-Cafe_Ache-Guanajuato_Central_Mexico_and_Gulf_Coast.html
188 [1250985, 'Café Aché', ['$15 - $50', 'Caribeña, Café, Cubana'], 5, 3, 'Mendizábal No. 23, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7646923-Reviews-Antigua_Casa_de_Musica-Guanajuato_Central_Mexico_and_Gulf_Coast.html
189 [7646923, 'Antigua Casa de Musica', ['Mexicana'], 2, 5, 'No. 2 Sangre de Cristo Centro, Guanajuato México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d12097229-Reviews-Pizza_Ilimitada_Queridapizza-Guanajuato_Central_Mexico_and_Gulf_Coast.html
190 [12097229, 'Pizza Ilimitada Queridapizza', ['$90 - $250', 'Italiana, Mexicana, Bar, Pizzería, Café, Pub', 'Almuerzo, Cena, Bebidas'], 7, 3, 'Avenida Juárez 9 Planta baja, Guanajuato 36000 México']


216 [10678386, 'A la Mexicana', ['Almuerzo, Cena'], 2, 4, 'Plaza San Javier No. 1, Guanajuato 36020 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g151932-d23014476-Reviews-Ruby_Joy_s_Delivery_Only_Restaurant-San_Miguel_de_Allende_Central_Mexico_and_Gul.html
217 [23014476, "Ruby Joy's Delivery Only Restaurant", ['$119 - $298', 'Contemporánea, Saludable, Internacional, Delicatessen, Fusión', 'Almuerzo, Cena'], 2, 5, 'Delivery Only, San Miguel de Allende 37796 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d6878204-Reviews-Bar_Esas_Ruinas-Guanajuato_Central_Mexico_and_Gulf_Coast.html
218 [6878204, 'Bar Esas Ruinas', ['Mexicana'], 2, 4, 'Carretera Guanajuato - Juventino Rosas Kilómetro 6.5 Centro, Guanajuato México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d8751194-Reviews-Mi_Palapa_Guanajuato-Guanajuato_Central_Mexico_and_Gulf_Coast.html
219 [8751194, 'Mi  Palapa Guanajuato', ['$100 - $280', 'Mexicana, Mariscos, Internacional, Gastropub'], 1

246 [21023819, 'Bartola Sky Bar', ['$60 - $300', 'Bebidas', 'Asientos al aire libre, Sirve alcohol, Vino y cerveza'], 3, 3, 'Positos 33, Guanajuato 36000 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d2328680-Reviews-Restaurante_Bar_Tradiciones_Panoramico-Guanajuato_Central_Mexico_and_Gulf_Coast.html
247 [232868, 'Restaurante Bar Tradiciones Panoramico', ['Internacional'], 1, 2, 'Hotel Guanajuato, Guanajuato México']
https://www.tripadvisor.com.mx/Restaurant_Review-g151932-d23014476-Reviews-Ruby_Joy_s_Delivery_Only_Restaurant-San_Miguel_de_Allende_Central_Mexico_and_Gul.html
248 [23014476, "Ruby Joy's Delivery Only Restaurant", ['$119 - $298', 'Contemporánea, Saludable, Internacional, Delicatessen, Fusión', 'Almuerzo, Cena'], 2, 5, 'Delivery Only, San Miguel de Allende 37796 México']
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d3892897-Reviews-Van_Gogh_Restaurant_Bar_Cafe-Guanajuato_Central_Mexico_and_Gulf_Coast.html
249 [3892897, 'Van Gogh Restaurant B

AttributeError: 'NoneType' object has no attribute 'get_text'

Unfortunately we could not retrieve all the restaurants, out the 325 reported restaurants in TripAdvisor located in Guanajuato, this is because the way we did the scraper sometimes the conexion gets lost and the script no longer works. This clearly, can be improved. 

The following code is to retrive the reviews, in which case we make use of the library Selenium. The choice of this was to be able to click on the "more" for each review so that the whole review is captured, however as this a dynamica process the time to scrapp takes longer. Instead, we choose to retrive what we could, that is, without clicking. The code can be used to scrapp the whole review but has to be improved to go through all the pages (review pages) for each restaurant.

In [2]:
import time
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Downloading: 16.2kB [00:00, 3.94MB/s]                                                                                                                                                                                        
[WDM] - Downloading: 16.2kB [00:00, 4.49MB/s]                                                                                                                                                                                        
[WDM] - Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.59M/2.59M [00:00<00:00, 10.6MB/s]
/tmp/ipykernel_18342/2304432791.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [3]:
def get_ids_reviews(bsop):
    r_id=[]
    for x in bsop.findAll('div',{'class':'prw_rup prw_reviews_review_resp'}):
        aux=x.find('div',{'class':'reviewSelector'})
        r_id.append(aux['data-reviewid'])
    return r_id

def get_reviews(r_id,url):
    driver.get(url)
    rev=[]
    for idx in r_id:
        time.sleep(2)
        review = driver.find_element("id",'review_'+idx ) 
        #review.find_element("class name", "entry").
        review.find_element("tag name","p")
        time.sleep(1)
        #review.find_element("tag name","span")
        #review.find_element("class name","taLnk.ulBlueLinks").click()
        new_review = driver.find_element( 'xpath','(//*[@id="review_'+idx+'"]//p)')
        rev.append(new_review.text)
    return rev

The file to write and save the reviews

In [4]:
path_to_file = "./restaurants_rev.csv"
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

In [7]:
def get_reviews_page(page):
    headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    html=requests.get(page, headers=headers)
    bsop=soup(html.content,'lxml')
    rv_ids=get_ids_reviews(bsop)
    print(rv_ids)
    print(get_reviews(rv_ids,page))

In [8]:
i=0
for p in ['10', '40', '70']:
    sleep(randint(1,5))
    url_='https://www.tripadvisor.com.mx/RestaurantSearch-g150799-oa'+p+'-Guanajuato_Central_Mexico_and_Gulf_Coast.html#EATERY_LIST_CONTENTS'
    headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    html=requests.get(url_, headers=headers)
    bsop=soup(html.content,'lxml')
    links=get_links_restaurants(bsop)
    for link in links:
        print(link)
        #call the server
        idx=int(re.findall(r'\d+[1-9]+',link)[1])
        headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
        html_temp=requests.get(link, headers=headers)
        sleep(randint(1,5)) #so we don't get blocked
        bsop_temp=soup(html_temp.content,'lxml')
        
        #get_the_name_of_restaurant
        name_res=bsop_temp.find('h1',{'data-test-target':'top-info-header'}).get_text()
        
        #scraping reviews
        rv_ids=get_ids_reviews(bsop_temp)
        list_rev=get_reviews(rv_ids,link)
        reviews=' '.join(rev for rev in list_rev)
        #list_temp=[rev for rev in list_temp]
        #print(name_res,' '.join(rev for rev in list_rev))
        csvWriter.writerow([idx,name_res,reviews])

https://www.tripadvisor.com.mx/Restaurant_Review-g151932-d23014476-Reviews-Ruby_Joy_s_Delivery_Only_Restaurant-San_Miguel_de_Allende_Central_Mexico_and_Gul.html
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d7291750-Reviews-La_Vie_en_Rose-Guanajuato_Central_Mexico_and_Gulf_Coast.html
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d2052606-Reviews-Santo_Cafe-Guanajuato_Central_Mexico_and_Gulf_Coast.html
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d866744-Reviews-La_Capellina-Guanajuato_Central_Mexico_and_Gulf_Coast.html
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d1986170-Reviews-La_Trattoria_de_Elena-Guanajuato_Central_Mexico_and_Gulf_Coast.html
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d14215661-Reviews-Los_Huacales-Guanajuato_Central_Mexico_and_Gulf_Coast.html
https://www.tripadvisor.com.mx/Restaurant_Review-g150799-d4039593-Reviews-Mercado_Hidalgo-Guanajuato_Central_Mexico_and_Gulf_Coast.html
https://www.tripadvisor.com.m

KeyboardInterrupt: 